<a href="https://colab.research.google.com/github/lwerth/INFO510-public/blob/master/info510_hmwk4_question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regression with many explanatory variables: Table 15.2 displays data from a designed experiment for a chemical process. In using these data to illustrate various approaches to selection and estimation of regression coefficients, Marquardt and Snee (1975) assume a quadratic regression form; that is, a linear relation between the expectation of the untransformed outcome, y, and the variables $x_1, x_2, x_3$, their two-way interactions, $x_1x_2, x_1x_3, x_2x_3$, and their squares, $x_2^{1}, x_2^{2}, x_2^{3}$.

## 3
(a) Fit an ordinary linear regression model (that is, nonhierarchical with a uniform prior distribution on the coefficients), including a constant term and the nine explanatory variables above.

In [3]:
# ! pip install pymc3 --upgrade
# ! pip install arviz

In [4]:
import numpy as np, scipy as sp, pymc3 as pm, matplotlib.pyplot as plt, pandas as pd, theano, theano.tensor as tt
from scipy import stats
import seaborn as sb
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import arviz as az

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

import theano

In [5]:
chemical = pd.read_csv('/content/drive/My Drive/Fall 2020/info510/hwk4/chemical.csv')
chemical.head()

chem_blocks = chemical.x1.unique()
chem_idx = OrdinalEncoder(dtype = np.int64).fit_transform(chemical[['x1']])
n_chem = len(chemical.x1.unique())

chemical.head()

,x1,x2,x3,y
0,1300,7.5,0.0120,49.0
1,1300,9.0,0.0120,50.2
2,1300,11.0,0.0115,50.5
3,1300,13.5,0.0130,48.5
4,1300,17.0,0.0135,47.5


In [34]:
#create a data frame with the 9 explanatory variables

variables = chemical.drop('y', axis=1)
variables['x1x2'] = variables.x1.multiply(variables.x2)
variables['x1x3'] = variables.x1.multiply(variables.x3)
variables['x2x3'] = variables.x2.multiply(variables.x3)
variables['x1_2'] = variables.x1**2
variables['x2_2'] = variables.x2**2
variables['x3_2'] = variables.x3**2
variables

#create another dataframe of the 9 explanatory variables and the outcome, y
chem_var = variables.join(chemical.y)
n_var = len(chem_var.y.unique())
chem_var


,x1,x2,x3,x1x2,x1x3,x2x3,x1_2,x2_2,x3_2,y
0,1300,7.5,0.0120,9750.0,15.60,0.0900,1690000,56.25,0.000144,49.0
1,1300,9.0,0.0120,11700.0,15.60,0.1080,1690000,81.00,0.000144,50.2
2,1300,11.0,0.0115,14300.0,14.95,0.1265,1690000,121.00,0.000132,50.5
3,1300,13.5,0.0130,17550.0,16.90,0.1755,1690000,182.25,0.000169,48.5
4,1300,17.0,0.0135,22100.0,17.55,0.2295,1690000,289.00,0.000182,47.5
5,1300,23.0,0.0120,29900.0,15.60,0.2760,1690000,529.00,0.000144,44.5
6,1200,5.3,0.0400,6360.0,48.00,0.2120,1440000,28.09,0.001600,28.0
7,1200,7.5,0.0380,9000.0,45.60,0.2850,1440000,56.25,0.001444,31.5
8,1200,11.0,0.0320,13200.0,38.40,0.3520,1440000,121.00,0.001024,34.5
9,1200,13.5,0.0260,16200.0,31.20,0.3510,1440000,182.25,0.000676,35.0


In [82]:
with pm.Model() as ord_model:
  alpha = pm.Normal('alpha', 0, 1)
  sigma = pm.HalfCauchy('sigma', 5)

  vari = theano.shared(chem_var.loc[:,'x1':'x3_2'].values)

  b = pm.Flat('b', shape =9)

  mu = alpha + b[0]*chem_var.x1 + b[1]*chem_var.x2 + b[2]*chem_var.x3 + b[3]*chem_var.x1x2 +  b[4]*chem_var.x1x3 + b[5]*chem_var.x2x3 + b[6]*chem_var.x1_2 + b[7]*chem_var.x2_2 + b[8]*chem_var.x3_2

  y = pm.Normal('y', mu, sigma, observed=chem_var.y)




  # mu = alpha + b[chem_var.x1] + b[chem_var.x2] + b[chem_var.x3] + b[chem_var.x1x2] + b[chem_var.x1x3] + b[chem_var.x2x3] + b[chem_var.x1_2] + b[chem_var.x2_2] + b[chem_var.x3_2]

print(mu)

  # b_x1 = pm.Normal('b_x1', 0, 1, shape = n_var)
  # b_x2 = pm.Normal('b_x2', 0, 1, shape = n_var)
  # b_x3 = pm.Normal('b_x3', 0, 1, shape = n_var)
  # b_x1x2 = pm.Normal('b_x1x2', 0, 1, shape = n_var)
  # b_x1x3 = pm.Normal('b_x1x3', 0, 1, shape = n_var)
  # b_x2x3 = pm.Normal('b_x2x3', 0, 1, shape = n_var)
  # b_x1_2 = pm.Normal('b_x1_2', 0, 1, shape = n_var)
  # b_x2_2 = pm.Normal('b_x2_2', 0, 1, shape = n_var)
  # b_x3_2 = pm.Normal('b_x3_2', 0, 1, shape = n_var)

  # mu = alpha + tt.dot(b_x1, chem_var.x1) + tt.dot(b_x2, chem_var.x2)
  # + tt.dot(b_x3, chem_var.x3) + tt.dot(b_x1x2, chem_var.x1x2) + tt.dot(b_x1x3, chem_var.x1x3)
  # + tt.dot(b_x2x3, chem_var.x2x3) +tt.dot(b_x1_2, chem_var['x1^2']) +tt.dot(b_x2_2, chem_var['x2^2'])
  # + tt.dot(b_x3_2, chem_var['x3^2'])



Elemwise{add,no_inplace}.0


In [83]:
with ord_model:
    trace_1 = pm.sample(2000)
    trace_1_sum = pm.summary(trace_1)

trace_1_sum

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [b, sigma, alpha]


ValueError: ignored

In [4]:
with pm.Model() as model_1:

    # Independent parameters for each county
    a = pm.Normal('a', 0, sigma=100, shape=n_chem)
    b = pm.Normal('b', 0, sigma=100, shape=n_chem)

    # Model error
    eps = pm.HalfCauchy('eps', 5)

    # Model prediction of radon level
    # a[county_idx] translates to a[0, 0, 0, 1, 1, ...],
    # we thus link multiple household measures of a county
    # to its coefficients.
    chem_est = a[chem_idx] + b[chem_idx]*chemical.y.values

    # Data likelihood
    y = pm.Normal('y', chem_est, sigma=eps, observed=chemical.y)

In [7]:
# with model_1:
#     trace_1 = pm.sample(2000)
#     trace_1_sum = pm.summary(trace_1)

# trace_1_sum

In [48]:
with pm.Model() as unpooled_model:

    # Independent parameters for each block
    ba = pm.Normal('ba', 0, sigma=100, shape=n_pen)
    bb = pm.Normal('bb', 0, sigma=100, shape=n_pen)
    bc = pm.Normal('bc', 0, sigma=100, shape=n_pen)
    bd = pm.Normal('bd', 0, sigma=100, shape=n_pen)


    # Model error
    eps = pm.HalfCauchy('eps', 5)

    # Model prediction of treatments level
    # a[county_idx] translates to a[0, 0, 0, 1, 1, ...],
    # we thus link multiple household measures of a county to its coefficients.
    treat_est = ba[pen_idx] + bb[pen_idx] + bc[pen_idx]*penecillin.D.values

    # Data likelihood
    y = pm.Normal('y', treat_est, sigma=eps, observed=penecillin.mean)



# Fully pooled; simple logistic regression

# with pm.Model() as log_model:
#     a = pm.Normal('a', 0, 2, shape = n_pen)
#     b = pm.Normal('b', 0, 1, shape = n_pen)
#     sigma = pm.HalfCauchy('sigma', 5)
#     mu = pm.math.invlogit(a + b[pen_idx] * pen_idx)
#     score = pm.Normal('score', mu=mu, observed = "penecillin.D, penecillin.C, penecillin.B",)

    trace_simple = pm.sample(2000, target_accept = 0.95, tune = 1000)
    summary_simple = pm.summary(trace_simple)

TypeError: ignored